In [1]:

from pathlib import Path
import numpy as np
import random
from collections import Counter
from omnibelt import load_json


In [2]:

root = Path('../data/backup')
path = root / 'cladder-v1-q-aggregate.json'
path = root / 'cladder-v1-q-commonsense.json'
path = root / 'cladder-v1-q-balanced.json'
# path = '../data/cladder-v1-common-easy.json'
full = load_json(path)
full_ids = {entry['question_id']: entry for entry in full}
models = load_json(root / 'cladder-v1-meta-models.json')
model_table = {info['model_id']: info for info in models}
len(full), len(models)

(10112, 7064)

In [3]:
# models[1]

In [4]:
full[8003]

{'question_id': 25722,
 'desc_id': 'nonsense1-mediation-nde-model6194-spec0-q1',
 'given_info': 'For those who are not kwox and are not swoq, the probability of kwoz is 24%. For those who are not kwox and are swoq, the probability of kwoz is 15%. For those who are kwox and are not swoq, the probability of kwoz is 8%. For those who are kwox and are swoq, the probability of kwoz is 51%. For those who are not kwox, the probability of swoq is 12%. For those who are kwox, the probability of swoq is 50%.',
 'question': 'If we disregard the mediation effect through swoq, would kwox negatively affect kwoz?',
 'answer': 'yes',
 'meta': {'story_id': 'nonsense1',
  'graph_id': 'mediation',
  'mediators': ['V2'],
  'polarity': False,
  'groundtruth': -0.10102520089737221,
  'query_type': 'nde',
  'rung': 3,
  'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]',
  'given_info': {'p(Y | X, V2)': [[0.2437127161150832, 0.154040346732467],
    [0.07985903371822341, 0.5100300185877629]],
   'p(V2 | X)': [

In [5]:
key = 'rung'
vocabs = {}
for entry in full:
	vocabs.setdefault(entry['meta'][key], []).extend(entry['question'].split())
	vocabs.setdefault(entry['meta'][key], []).extend(entry['given_info'].split())
	model = models[entry['meta']['model_id']]
	vocabs.setdefault(entry['meta'][key], []).extend(model['background'].split())
def _cleanup(w):
	w = w.lower()
	w = w.replace('?', '').replace('.', '').replace(',', '').replace('\'', '').replace('\"', '').replace('%', '').replace(':', '')
	# check if it's a number
	try:
		float(w)
		return 'NUM'
	except:
		pass
	return w
vocabs = {k: [_cleanup(w) for w in v] for k, v in vocabs.items()}
vocabs = {k: Counter(v) for k, v in vocabs.items()}
vocabs.keys()

dict_keys([3, 2, 1])

In [6]:
# vocabs

In [8]:
{k: len(v) for k, v in vocabs.items()}

{3: 479, 2: 400, 1: 405}

In [10]:
random.seed(0)
indices = random.choices(list(range(len(full))), k=50)
print(indices)
assert len(set(indices)) == len(indices)

[8538, 7664, 4252, 2618, 5170, 4094, 7925, 3067, 4819, 5899, 9182, 5103, 2849, 7642, 6252, 2533, 9199, 9937, 8192, 9122, 3136, 7380, 9089, 6916, 4774, 1018, 4390, 6177, 9232, 9774, 4823, 8750, 2634, 8140, 5548, 141, 7277, 4032, 8340, 6756, 11, 4991, 8773, 2466, 3288, 8802, 1932, 5738, 2412, 9783]


In [16]:
for i, ind in enumerate(indices):
	entry = full[ind]
	model = models[entry['meta']['model_id']]
	print(i+1)
	print(model['background'])
	print(entry['given_info'])
	print(entry['question'])
	# print(entry['answer'])
	print()

1
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Hwax has a direct effect on gyzp and jyka. Jyka has a direct effect on lirg. Gyzp has a direct effect on lirg.
The overall probability of jyka is 18%. For those who are not jyka, the probability of lirg is 72%. For those who are jyka, the probability of lirg is 35%.
Is lirg less likely than not lirg overall?

2
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Hwax has a direct effect on kraz and pexu. Pexu has a direct effect on rukz. Kraz has a direct effect on rukz.
For those who are not pexu and are not kraz, the probability of rukz is 54%. For those who are not pexu and are kraz, the probability of rukz is 27%. For those who are pexu and are not kraz, the probability of rukz is 11%. For those who are pexu and are kraz, the probability of rukz is 96%

In [13]:
len([e for e in full if e['desc_id'].startswith('nonse')])/len(full)

0.37994462025316456